In [2]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, unquote
import os
import time

# 配置参数
BASE_URL = "https://www.etit.kit.edu/"
START_URL = urljoin(BASE_URL, "vertiefungsrichtungen_master.php")
PDF_ROOT = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "de-DE,de;q=0.9"
}
REQUEST_DELAY = 1  # 请求间隔

# 定义目标PDF标题（德语原文）
TARGET_LINKS = {
    "exemplarischer": {
        "pattern": re.compile(r'^Exemplarischer Studienplan\b', re.I),
        "filename": "Exemplarischer_Studienplan.pdf"
    },
    "individueller": {
        "pattern": re.compile(r'^Individueller Studienplan\b.*ab WS 2018/19', re.I),
        "filename": "Individueller_Studienplan.pdf"
    },
    "empfohlene": {
        "pattern": re.compile(r'^Empfohlene Wahlmodule\b', re.I),
        "filename": "Empfohlene_Wahlmodule.pdf"
    }
}

def create_directory(dir_path):
    """创建存储目录"""
    try:
        os.makedirs(dir_path, exist_ok=True)
        print(f"创建目录：{dir_path}")
    except Exception as e:
        print(f"目录创建失败：{str(e)}")
        return False
    return True

def get_final_pdf_url(link_url):
    """获取最终PDF地址（处理重定向）"""
    try:
        with requests.Session() as session:
            response = session.head(link_url, headers=HEADERS, allow_redirects=True, timeout=10)
            response.raise_for_status()
            return response.url
    except Exception as e:
        print(f"解析PDF地址失败：{str(e)}")
        return None

def process_direction(direction_url, dir_number):
    """处理单个专业方向"""
    print(f"\n{'='*40}\n正在处理方向：{dir_number}")
    
    try:
        # 创建存储目录
        dir_path = os.path.join(PDF_ROOT, f"vertiefungsrichtung_{dir_number}")
        if not create_directory(dir_path):
            return

        # 获取方向页面
        response = requests.get(direction_url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 查找所有包含PDF的链接
        pdf_links = {}
        for a_tag in soup.find_all('a', href=True):
            link_text = a_tag.get_text(strip=True)
            for key, config in TARGET_LINKS.items():
                if config["pattern"].search(link_text):
                    # 获取原始链接
                    raw_link = urljoin(direction_url, a_tag['href'])
                    # 获取最终PDF地址
                    pdf_url = get_final_pdf_url(raw_link)
                    if pdf_url and pdf_url.lower().endswith('.pdf'):
                        pdf_links[key] = {
                            "url": pdf_url,
                            "filename": config["filename"]
                        }
                    break

        # 下载所有找到的PDF
        download_count = 0
        for key, data in pdf_links.items():
            file_path = os.path.join(dir_path, data["filename"])
            if os.path.exists(file_path):
                print(f"文件已存在：{data['filename']}")
                continue

            print(f"正在下载：{data['filename']}")
            try:
                response = requests.get(data["url"], headers=HEADERS)
                response.raise_for_status()
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                download_count += 1
                time.sleep(REQUEST_DELAY)
            except Exception as e:
                print(f"下载失败：{str(e)}")

        print(f"完成下载：{download_count}/3 个文件")

    except Exception as e:
        print(f"处理方向时发生错误：{str(e)}")

def main():
    print("正在解析主页面...")
    try:
        # 获取所有方向链接
        response = requests.get(START_URL, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 匹配所有vertiefungsrichtung_数字.php的链接
        direction_links = []
        pattern = re.compile(r'vertiefungsrichtung_(\d+)\.php')
        for a_tag in soup.find_all('a', href=pattern):
            full_url = urljoin(BASE_URL, a_tag['href'])
            match = pattern.search(full_url)
            if match:
                direction_links.append( (full_url, match.group(1)) )  # (URL, 编号)

        # === 关键修改：有序去重 ===
        seen = set()
        unique_links = []
        for link in direction_links:
            # 用URL作为唯一标识
            if link[0] not in seen:  # link[0] 是完整URL
                seen.add(link[0])
                unique_links.append(link)
        direction_links = unique_links
        # =========================

        # direction_links = list(set(direction_links))  # 去重
        # 这个是无序去重，所以用上面的。

        total = len(direction_links)
        print(f"找到 {total} 个专业方向")

        # 遍历处理每个方向
        for idx, (url, number) in enumerate(direction_links, 1):
            print(f"\n▶ 进度：{idx}/{total}")
            process_direction(url, number)
        
        print("\n✅ 所有方向处理完成！")

    except Exception as e:
        print(f"主流程错误：{str(e)}")

if __name__ == "__main__":
    main()

正在解析主页面...
找到 23 个专业方向

▶ 进度：1/23

正在处理方向：2
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_2
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：2/23

正在处理方向：3
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_3
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：3/23

正在处理方向：4
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_4
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：4/23

正在处理方向：5
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_5
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：5/23

正在处理方向：6
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_6
文件已存在：Exemplarischer_Stud

In [4]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, unquote
import os
import time

# 配置参数
BASE_URL = "https://www.etit.kit.edu/"
START_URL = urljoin(BASE_URL, "vertiefungsrichtungen_master.php")
PDF_ROOT = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "de-DE,de;q=0.9"
}
REQUEST_DELAY = 1  # 请求间隔

# 定义目标PDF标题（德语原文）
TARGET_LINKS = {
    "exemplarischer": {
        "pattern": re.compile(r'^Exemplarischer Studienplan\b', re.I),
        "filename": "Exemplarischer_Studienplan.pdf"
    },
    "individueller": {
        "pattern": re.compile(r'^Individueller Studienplan\b.*ab WS 2018/19', re.I),
        "filename": "Individueller_Studienplan.pdf"
    },
    "empfohlene": {
        "pattern": re.compile(r'^Empfohlene Wahlmodule\b', re.I),
        "filename": "Empfohlene_Wahlmodule.pdf"
    }
}

def create_directory(dir_path):
    """创建存储目录"""
    try:
        os.makedirs(dir_path, exist_ok=True)
        print(f"创建目录：{dir_path}")
    except Exception as e:
        print(f"目录创建失败：{str(e)}")
        return False
    return True

def get_final_pdf_url(link_url):
    """获取最终PDF地址（处理重定向）"""
    try:
        with requests.Session() as session:
            response = session.head(link_url, headers=HEADERS, allow_redirects=True, timeout=10)
            response.raise_for_status()
            return response.url
    except Exception as e:
        print(f"解析PDF地址失败：{str(e)}")
        return None

def process_direction(direction_url, dir_number):
    """处理单个专业方向"""
    print(f"\n{'='*40}\n正在处理方向：{dir_number}")
    
    try:
        # 创建存储目录
        dir_path = os.path.join(PDF_ROOT, f"vertiefungsrichtung_{dir_number}")
        if not create_directory(dir_path):
            return

        # 获取方向页面
        response = requests.get(direction_url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 查找所有包含PDF的链接
        pdf_links = {}
        for a_tag in soup.find_all('a', href=True):
            link_text = a_tag.get_text(strip=True)
            for key, config in TARGET_LINKS.items():
                if config["pattern"].search(link_text):
                    # 获取原始链接
                    raw_link = urljoin(direction_url, a_tag['href'])
                    # 获取最终PDF地址
                    pdf_url = get_final_pdf_url(raw_link)
                    if pdf_url and pdf_url.lower().endswith('.pdf'):
                        pdf_links[key] = {
                            "url": pdf_url,
                            "filename": config["filename"]
                        }
                    break

        # 下载所有找到的PDF
        download_count = 0
        for key, data in pdf_links.items():
            file_path = os.path.join(dir_path, data["filename"])
            if os.path.exists(file_path):
                print(f"文件已存在：{data['filename']}")
                continue

            print(f"正在下载：{data['filename']}")
            try:
                response = requests.get(data["url"], headers=HEADERS)
                response.raise_for_status()
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                download_count += 1
                time.sleep(REQUEST_DELAY)
            except Exception as e:
                print(f"下载失败：{str(e)}")

        print(f"完成下载：{download_count}/3 个文件")

    except Exception as e:
        print(f"处理方向时发生错误：{str(e)}")

def get_direction_links():
    """获取所有专业方向页面链接"""
    print("正在解析主页面...")
    try:
        # 获取所有方向链接
        response = requests.get(START_URL, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 匹配所有vertiefungsrichtung_数字.php的链接
        direction_links = []
        pattern = re.compile(r'vertiefungsrichtung_(\d+)\.php')
        for a_tag in soup.find_all('a', href=pattern):
            full_url = urljoin(BASE_URL, a_tag['href'])
            match = pattern.search(full_url)
            if match:
                direction_links.append( (full_url, match.group(1)) )  # (URL, 编号)

        # === 关键修改：有序去重 ===
        seen = set()
        unique_links = []
        for link in direction_links:
            # 用URL作为唯一标识
            if link[0] not in seen:  # link[0] 是完整URL
                seen.add(link[0])
                unique_links.append(link)
        direction_links = unique_links
        # =========================

        # direction_links = list(set(direction_links))  # 去重
        # 这个是无序去重，所以用上面的。
        return direction_links 

    except Exception as e:
        print(f"获取方向列表失败: {str(e)}")
        return []

def main():
    try:
        direction_links = get_direction_links()
        total = len(direction_links)
        print(f"找到 {total} 个专业方向")

        # 遍历处理每个方向
        for idx, (url, number) in enumerate(direction_links, 1):
            print(f"\n▶ 进度：{idx}/{total}")
            process_direction(url, number)
        
        print("\n✅ 所有方向处理完成！")

    except Exception as e:
        print(f"主流程错误：{str(e)}")

if __name__ == "__main__":
    main()

正在解析主页面...
找到 23 个专业方向

▶ 进度：1/23

正在处理方向：2
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_2
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：2/23

正在处理方向：3
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_3
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：3/23

正在处理方向：4
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_4
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：4/23

正在处理方向：5
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_5
文件已存在：Exemplarischer_Studienplan.pdf
文件已存在：Individueller_Studienplan.pdf
文件已存在：Empfohlene_Wahlmodule.pdf
完成下载：0/3 个文件

▶ 进度：5/23

正在处理方向：6
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_6
文件已存在：Exemplarischer_Stud

In [6]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, unquote
import os
import time

# 配置参数 
BASE_URL = "https://www.etit.kit.edu/"
START_URL = urljoin(BASE_URL, "vertiefungsrichtungen_master.php")
PDF_ROOT = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "de-DE,de;q=0.9"
}
REQUEST_DELAY = 1  # 请求间隔

# 定义目标PDF标题（德语原文）
# 改进后的多语言配置 英文有5个方向是英语的，所以其字段也不一样
TARGET_LINKS = {
    "exemplary": {
        "pattern": re.compile(
            r'^(Exemplarischer\s+Studienplan|Exemplary\s+Curriculum)\b', 
            re.I | re.U
        ),
        "filename": "Exemplary_Curriculum.pdf"
    },
    "individual": {
        "pattern": re.compile(
            r'^(Individueller\s+Studienplan|Individual\s+Study\s+Plan)\b.*(ab WS 2018/19|starting from winter semester 2018/19)',
            re.I | re.U
        ),
        "filename": "Individual_Study_Plan.pdf"
    },
    "elective": {
        "pattern": re.compile(
            r'^(Empfohlene\s+Wahlmodule|Recommended\s+Elective\s+Modules)\b',
            re.I | re.U
        ),
        "filename": "Recommended_Elective_Modules.pdf"
    }
}

def create_directory(dir_path):
    """创建存储目录"""
    try:
        os.makedirs(dir_path, exist_ok=True)
        print(f"创建目录：{dir_path}")
    except Exception as e:
        print(f"目录创建失败：{str(e)}")
        return False
    return True

def get_final_pdf_url(link_url):
    """获取最终PDF地址（处理重定向）"""
    try:
        with requests.Session() as session:
            response = session.head(link_url, headers=HEADERS, allow_redirects=True, timeout=10)
            response.raise_for_status()
            return response.url
    except Exception as e:
        print(f"解析PDF地址失败：{str(e)}")
        return None

def process_direction(direction_url, dir_number):
    """处理单个专业方向"""
    print(f"\n{'='*40}\n正在处理方向：{dir_number}")
    
    try:
        # 创建存储目录
        dir_path = os.path.join(PDF_ROOT, f"vertiefungsrichtung_{dir_number}")
        if not create_directory(dir_path):
            return

        # 获取方向页面
        response = requests.get(direction_url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 查找所有包含PDF的链接
        pdf_links = {}
        for a_tag in soup.find_all('a', href=True):
            link_text = a_tag.get_text(strip=True)
            for key, config in TARGET_LINKS.items():
                if config["pattern"].search(link_text):
                    # 获取原始链接
                    raw_link = urljoin(direction_url, a_tag['href'])
                    # 获取最终PDF地址
                    pdf_url = get_final_pdf_url(raw_link)
                    if pdf_url and pdf_url.lower().endswith('.pdf'):
                        pdf_links[key] = {
                            "url": pdf_url,
                            "filename": config["filename"]
                        }
                    break

        # 下载所有找到的PDF
        download_count = 0
        for key, data in pdf_links.items():
            file_path = os.path.join(dir_path, data["filename"])
            if os.path.exists(file_path):
                print(f"文件已存在：{data['filename']}")
                continue

            print(f"正在下载：{data['filename']}")
            try:
                response = requests.get(data["url"], headers=HEADERS)
                response.raise_for_status()
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                download_count += 1
                time.sleep(REQUEST_DELAY)
            except Exception as e:
                print(f"下载失败：{str(e)}")

        print(f"完成下载：{download_count}/3 个文件")

    except Exception as e:
        print(f"处理方向时发生错误：{str(e)}")

def get_direction_links():
    """获取所有专业方向页面链接"""
    print("正在解析主页面...")
    try:
        # 获取所有方向链接
        response = requests.get(START_URL, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 匹配所有vertiefungsrichtung_数字.php的链接
        direction_links = []
        #加上, re.I表示不管大小写。因为方向25是后面加上去的其href为Vertiefungsrichtung_25。傻逼KIT属实不严谨
        pattern = re.compile(r'vertiefungsrichtung_(\d+)\.php', re.I)      
        for a_tag in soup.find_all('a', href=pattern):
            full_url = urljoin(BASE_URL, a_tag['href'])
            match = pattern.search(full_url)
            if match:
                direction_links.append( (full_url, match.group(1)) )  # (URL, 编号)

        # === 关键修改：有序去重 ===
        seen = set()
        unique_links = []
        for link in direction_links:
            # 用URL作为唯一标识
            if link[0] not in seen:  # link[0] 是完整URL
                seen.add(link[0])
                unique_links.append(link)
        direction_links = unique_links
        # =========================

        # direction_links = list(set(direction_links))  # 去重
        # 这个是无序去重，所以用上面的。
        return direction_links 

    except Exception as e:
        print(f"获取方向列表失败: {str(e)}")
        return []

def main():
    try:
        direction_links = get_direction_links()
        total = len(direction_links)
        print(f"找到 {total} 个专业方向")

        # 遍历处理每个方向
        for idx, (url, number) in enumerate(direction_links, 1):
            print(f"\n▶ 进度：{idx}/{total}")
            process_direction(url, number)
        
        print("\n✅ 所有方向处理完成！")

    except Exception as e:
        print(f"主流程错误：{str(e)}")

if __name__ == "__main__":
    main()

正在解析主页面...
找到 24 个专业方向

▶ 进度：1/24

正在处理方向：2
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_2
文件已存在：Exemplary_Curriculum.pdf
文件已存在：Individual_Study_Plan.pdf
文件已存在：Recommended_Elective_Modules.pdf
完成下载：0/3 个文件

▶ 进度：2/24

正在处理方向：3
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_3
文件已存在：Exemplary_Curriculum.pdf
文件已存在：Individual_Study_Plan.pdf
文件已存在：Recommended_Elective_Modules.pdf
完成下载：0/3 个文件

▶ 进度：3/24

正在处理方向：4
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_4
文件已存在：Exemplary_Curriculum.pdf
文件已存在：Individual_Study_Plan.pdf
文件已存在：Recommended_Elective_Modules.pdf
完成下载：0/3 个文件

▶ 进度：4/24

正在处理方向：5
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_5
文件已存在：Exemplary_Curriculum.pdf
文件已存在：Individual_Study_Plan.pdf
文件已存在：Recommended_Elective_Modules.pdf
完成下载：0/3 个文件

▶ 进度：5/24

正在处理方向：6
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_6
文件已存在：Exemplary_Curriculum.pdf
文件已存在：

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, unquote
import os
import time

# 配置参数 大写是惯例
BASE_URL = "https://www.etit.kit.edu/"
START_URL = urljoin(BASE_URL, "vertiefungsrichtungen_master.php")
PDF_ROOT = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "de-DE,de;q=0.9"
}
REQUEST_DELAY = 1  # 请求间隔

# 定义目标PDF标题（德语原文）
# 改进后的多语言配置 英文有5个方向是英语的，所以其字段也不一样
TARGET_LINKS = {
    "exemplary": {
        "pattern": re.compile(
            r'^(Exemplarischer\s+Studienplan|Exemplary\s+Curriculum)\b', 
            re.I | re.U
        ),
        "filename": "Exemplary_Curriculum.pdf"
    },
    "individual": {
        "pattern": re.compile(
            r'^(Individueller\s+Studienplan|Individual\s+Study\s+Plan)\b.*(ab WS 2018/19|starting from winter semester 2018/19)',
            re.I | re.U
        ),
        "filename": "Individual_Study_Plan.pdf"
    },
    "elective": {
        "pattern": re.compile(
            r'^(Empfohlene\s+Wahlmodule|Recommended\s+Elective\s+Modules)\b',
            re.I | re.U
        ),
        "filename": "Recommended_Elective_Modules.pdf"
    }
}

def create_directory(dir_path):
    """创建存储目录"""
    try:
        os.makedirs(dir_path, exist_ok=True)
        print(f"创建目录：{dir_path}")
    except Exception as e:
        print(f"目录创建失败：{str(e)}")
        return False
    return True

def get_final_pdf_url(link_url):
    """获取最终PDF地址（处理重定向）"""
    try:
        with requests.Session() as session:
            response = session.head(link_url, headers=HEADERS, allow_redirects=True, timeout=10)
            response.raise_for_status()
            return response.url
    except Exception as e:
        print(f"解析PDF地址失败：{str(e)}")
        return None

# 专业层， 第二层
def process_direction(direction_url, dir_number):
    """处理单个专业方向"""
    print(f"\n{'='*40}\n正在处理方向：{dir_number}")
    
    try:

        # === 关键修改2：添加专业名字 ====
        # 我修改了顺序，先进入了专业的页面，在提取h1头标题，再创建文件夹
        # 获取方向页面
        response = requests.get(direction_url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        h1_tag = soup.find('h1')
        dir_name = h1_tag.get_text().split(':')[-1].strip() if h1_tag else f"Unnamed_{dir_number}"

        # 创建存储目录
        dir_path = os.path.join(PDF_ROOT, f"vertiefungsrichtung_{dir_number}_{dir_name}")
        if not create_directory(dir_path):
            return
        # ===============================

        # 查找所有包含PDF的链接
        pdf_links = {}
        for a_tag in soup.find_all('a', href=True):
            link_text = a_tag.get_text(strip=True)
            for key, config in TARGET_LINKS.items():
                if config["pattern"].search(link_text):
                    # 获取原始链接
                    raw_link = urljoin(direction_url, a_tag['href'])
                    # 获取最终PDF地址
                    pdf_url = get_final_pdf_url(raw_link)
                    if pdf_url and pdf_url.lower().endswith('.pdf'):
                        pdf_links[key] = {
                            "url": pdf_url,
                            "filename": config["filename"]
                        }
                    break

        # 下载所有找到的PDF
        download_count = 0
        for key, data in pdf_links.items():
            file_path = os.path.join(dir_path, data["filename"])
            if os.path.exists(file_path):
                print(f"文件已存在：{data['filename']}")
                continue

            print(f"正在下载：{data['filename']}")
            # 跳转层, 第三层，就是pdf的php链接和对应链接不符合
            try:
                response = requests.get(data["url"], headers=HEADERS)
                response.raise_for_status()
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                download_count += 1
                time.sleep(REQUEST_DELAY)
            except Exception as e:
                print(f"下载失败：{str(e)}")

        print(f"完成下载：{download_count}/3 个文件")

    except Exception as e:
        print(f"处理方向时发生错误：{str(e)}")

# 主页层。第一层
# 这个还在主页面，而添加专业的名字到文件夹需要进入方向内，用h1确定页面的主标题，才能提取出来，所以得在进图专业页面之后进行也就不是这里
# 整个文件的两层也就是三格个response，多了就冗余了。一个在主界面，一个在专业界面，一个负责找到pdf文件之后重定向
def get_direction_links():
    """获取所有专业方向页面链接"""
    print("正在解析主页面...")
    try:
        # 获取所有方向链接
        response = requests.get(START_URL, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 匹配所有vertiefungsrichtung_数字.php的链接
        direction_links = []
        #加上, re.I表示不管大小写。因为方向25是后面加上去的其href为Vertiefungsrichtung_25。傻逼KIT属实不严谨
        pattern = re.compile(r'vertiefungsrichtung_(\d+)\.php', re.I)      
        for a_tag in soup.find_all('a', href=pattern):
            full_url = urljoin(BASE_URL, a_tag['href'])
            match = pattern.search(full_url)
            if match:
                direction_links.append( (full_url, match.group(1)) )  # (URL, 编号)。 group(1)：第一个捕获组内容（即 \d+ 匹配的数字）

        # === 关键修改：有序去重 ===
        seen = set()
        unique_links = []
        for link in direction_links:
            # 用URL作为唯一标识
            if link[0] not in seen:  # link[0] 是完整URL
                seen.add(link[0])
                unique_links.append(link)
        direction_links = unique_links
        # =========================

        # direction_links = list(set(direction_links))  # 去重
        # 这个是无序去重，所以用上面的。
        return direction_links 

    except Exception as e:
        print(f"获取方向列表失败: {str(e)}")
        return []

def main():
    try:
        direction_links = get_direction_links()
        total = len(direction_links)
        print(f"找到 {total} 个专业方向")

        # 遍历处理每个方向
        for idx, (url, number) in enumerate(direction_links, 1):
            print(f"\n▶ 进度：{idx}/{total}")
            process_direction(url, number)

        print("\n✅ 所有方向处理完成！")

    except Exception as e:
        print(f"主流程错误：{str(e)}")

if __name__ == "__main__":
    main()

正在解析主页面...
找到 24 个专业方向

▶ 进度：1/24

正在处理方向：2
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_2_Signalverarbeitung
正在下载：Exemplary_Curriculum.pdf
正在下载：Individual_Study_Plan.pdf
正在下载：Recommended_Elective_Modules.pdf
完成下载：3/3 个文件

▶ 进度：2/24

正在处理方向：3
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_3_Biomedizinische Technik
正在下载：Exemplary_Curriculum.pdf
正在下载：Individual_Study_Plan.pdf
正在下载：Recommended_Elective_Modules.pdf
完成下载：3/3 个文件

▶ 进度：3/24

正在处理方向：4
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_4_Elektromobilität
正在下载：Exemplary_Curriculum.pdf
正在下载：Individual_Study_Plan.pdf
正在下载：Recommended_Elective_Modules.pdf
完成下载：3/3 个文件

▶ 进度：4/24

正在处理方向：5
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_5_Regelungs- und Steuerungstechnik
正在下载：Exemplary_Curriculum.pdf
正在下载：Individual_Study_Plan.pdf
正在下载：Recommended_Elective_Modules.pdf
完成下载：3/3 个文件

▶ 进度：5/24

正在处理方向：6
创建目录：E:\OneDrive - MSFT\.master

一、正则表达式捕获组解析
代码片段：

python
复制
pattern = re.compile(r'vertiefungsrichtung_(\d+)\.php', re.I)
match = pattern.search(full_url)
dir_number = match.group(1)
1. 捕获组 (group) 的概念
正则表达式中的括号 ( ) 表示一个"捕获组"，用于提取特定部分的匹配内容。

group(0) 总是代表完整匹配的整个字符串。

group(1) 表示第一个括号捕获的内容，group(2) 表示第二个括号捕获的内容，依此类推。

2. 您的正则表达式分析
python
复制
r'vertiefungsrichtung_(\d+)\.php'
结构分解：

vertiefungsrichtung_：固定匹配的文本

(\d+)：第一个捕获组，匹配1个或多个数字

\.php：匹配.php后缀（.需要转义）

示例：

python
复制
full_url = "https://.../vertiefungsrichtung_25.php"
match.group(0) # 返回 "vertiefungsrichtung_25.php"
match.group(1) # 返回 "25"
3. 多捕获组情景
若正则表达式为：

python
复制
r'(\d+)_(VR\d+)\.php'
匹配 "12_VR25.php" 时：

group(0) → "12_VR25.php"

group(1) → "12"

group(2) → "VR25"

二、BeautifulSoup 循环解析
代码片段：

python
复制
for a_tag in soup.find_all('a', href=pattern):
    full_url = urljoin(BASE_URL, a_tag['href'])
1. HTML <a> 标签基础
<a> 标签 是HTML中的超链接标签，典型结构：

html
复制
<a href="目标URL">链接文本</a>
运行 HTML
href 属性 存储链接的目标地址。

2. soup.find_all() 方法解析
作用：在HTML文档中搜索所有符合条件的标签。

参数 'a'：指定查找所有 <a> 标签。

参数 href=pattern：筛选 href 属性匹配正则表达式 pattern 的标签。

3. 循环过程解析
假设网页中有以下3个链接：

html
复制
<a href="vertiefungsrichtung_2.php">方向1</a>
<a href="invalid_link.html">无效链接</a>
<a href="Vertiefungsrichtung_25.php">方向2</a>
运行 HTML
循环流程：

第一轮循环：

a_tag 对应第一个 <a> 标签

a_tag['href'] 值为 "vertiefungsrichtung_2.php"

pattern.search() 匹配成功，进入处理逻辑

第二轮循环：

a_tag 对应第二个 <a> 标签

href 值为 "invalid_link.html"，不匹配 pattern，自动跳过

第三轮循环：

a_tag 对应第三个 <a> 标签

href 值为 "Vertiefungsrichtung_25.php"（因 re.I 忽略大小写，匹配成功）

三、代码执行流程图解
mermaid
复制
graph TD
    A[开始解析网页] --> B[查找所有<a>标签]
    B --> C{检查href是否匹配pattern?}
    C -->|匹配| D[提取href值]
    C -->|不匹配| E[跳过]
    D --> F[补全为完整URL]
    F --> G[正则提取编号]
    G --> H[存入direction_links列表]
    H --> I[循环下一个<a>标签]
    E --> I
四、常见问题解答
1. 为何要用 re.I 标志？
网站可能存在大小写不一致的URL（如 Vertiefungsrichtung_2.php 和 vertiefungsrichtung_2.php）。

re.I 使匹配不区分大小写，确保所有变体都能被捕获。

2. 如果网页中有重复链接怎么办？
示例代码中使用了有序去重逻辑：

python
复制
seen = set()
unique_links = []
for link in direction_links:
    if link[0] not in seen:
        seen.add(link[0])
        unique_links.append(link)
此逻辑会保留首次出现的链接，跳过后续重复项，同时保持原始顺序。

3. 如何验证匹配结果？
可在代码中添加调试语句：

python
复制
print("原始链接列表:")
for url, num in direction_links:
    print(f"编号 {num}: {url}")
五、关键总结
概念	说明
正则捕获组 ( )	用括号定义需提取的内容，group(1) 取第一个捕获组
soup.find_all()	根据标签名和属性过滤HTML元素
a_tag['href']	获取 <a> 标签的链接地址
urljoin()	将相对路径转换为绝对URL
有序去重	用 seen 集合记录已处理链接，保持原始顺序的同时去重
通过此代码，您可以精准抓取格式为 vertiefungsrichtung_数字.php 的有效链接，并提取出方向编号用于后续处理。

1. soup 对象
定义
soup 是通过 BeautifulSoup 库解析 HTML/XML 文档后生成的对象。

本质：它是一个复杂的树形数据结构，将网页内容转换为可编程操作的 Python 对象。

创建方法：

python
复制
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
其中：

html_content：网页的原始文本（例如 response.text）

"html.parser"：解析器类型（Python 内置解析器，其他选项如 lxml）

作用
提供便捷的方法搜索、遍历和修改 HTML 元素。

将复杂的 HTML 结构转换为可操作的树形结构。

示例
假设网页内容如下：

html
复制
<html>
  <body>
    <a href="page1.html">Link 1</a>
    <a href="page2.html">Link 2</a>
  </body>
</html>
运行 HTML
解析后，soup 对象允许你通过方法（如 find_all()）提取这些链接。

2. soup.find_all() 方法
定义
是 BeautifulSoup 对象的核心方法，用于搜索所有符合条件的 HTML 元素。

语法：

python
复制
soup.find_all(name, attrs, recursive, limit, **kwargs)
常用参数：

name：标签名（如 "a" 表示 <a> 标签）

attrs：属性过滤条件（如 href=re.compile(...)）

代码中 soup.find_all('a', href=pattern) 的含义
功能：查找所有 <a> 标签，且其 href 属性匹配正则表达式 pattern。

参数分解：

'a'：指定查找 <a> 标签（超链接标签）。

href=pattern：筛选 href 属性符合正则表达式 pattern 的标签。

返回值
返回一个 ResultSet 对象，本质是包含所有匹配标签的列表。

每个元素是一个 Tag 对象，表示一个 HTML 标签及其内容。

3. 代码逻辑详解
python
复制
for a_tag in soup.find_all('a', href=pattern):
    full_url = urljoin(BASE_URL, a_tag['href'])
    # ...
循环过程
遍历匹配的 <a> 标签：

soup.find_all('a', href=pattern) 返回所有符合条件的 <a> 标签。

a_tag 是循环变量，依次表示每个匹配的标签。

提取 href 属性：

a_tag['href'] 获取标签的 href 属性值（如 "vertiefungsrichtung_2.php"）。

补全 URL：

urljoin(BASE_URL, ...) 将相对路径转换为绝对 URL（如 https://.../vertiefungsrichtung_2.php）。

4. 关键概念总结
概念	说明
soup 对象	解析后的 HTML 文档树，提供结构化访问接口
find_all()	搜索文档中所有符合条件的标签
a_tag	表示单个 <a> 标签的 Tag 对象
a_tag['href']	获取标签的 href 属性值
ResultSet	find_all() 的返回值，类似列表，包含所有匹配的 Tag 对象
5. 为何需要这些操作？
网页数据抓取的核心步骤
获取原始 HTML：通过 requests.get() 下载网页。

结构化解析：用 BeautifulSoup 将 HTML 转换为可操作对象。

精准筛选：通过 find_all() 和正则表达式定位目标链接。

数据提取：从标签属性中提取关键信息（如 href）。

示例场景
假设网页中有以下链接：

html
复制
<a href="vertiefungsrichtung_2.php">方向1</a>
<a href="other_page.php">无效链接</a>
<a href="Vertiefungsrichtung_25.php">方向2</a>
运行 HTML
通过代码：

soup.find_all('a', href=pattern) 会匹配 第一个和第三个 <a> 标签。

循环中提取它们的 href，并补全为完整 URL。

6. 常见问题
Q1：为何 soup.find_all() 返回空列表？
可能原因：

网页结构已更新，目标标签不存在。

正则表达式 pattern 未正确匹配 href。

网页内容通过 JavaScript 动态加载（需改用 Selenium）。

Q2：如何处理大小写不一致的标签？
方法：在正则表达式中使用 re.I 标志（如代码中的 pattern），忽略大小写匹配。

Q3：a_tag 还能提取哪些信息？
其他常用操作：

python
复制
a_tag.text          # 获取链接文本（如 "方向1"）
a_tag.get('class')  # 获取 class 属性
a_tag.parent        # 获取父标签
通过以上解析，你可以理解代码如何通过 soup 和 find_all() 精准提取目标链接。这是网页抓取中“定位-提取”模式的核心实现。

一、pattern.search(full_url) 的作用
核心功能

pattern.search(full_url) 使用预定义的正则表达式模式 pattern 在完整的 URL (full_url) 中搜索匹配项。

如果匹配成功，返回一个 Match 对象；否则返回 None。

实际意义

验证 URL 格式：确保链接符合 vertiefungsrichtung_数字.php 的格式。

提取关键信息：通过正则的捕获组 (\d+) 提取方向编号。

示例

python
复制
full_url = "https://www.etit.kit.edu/Vertiefungsrichtung_25.php"  # 注意大小写
match = pattern.search(full_url)  # 匹配成功，返回 Match 对象
二、match.group(1) 的含义
正则捕获组

正则表达式 r'vertiefungsrichtung_(\d+)\.php' 中的 (\d+) 是第一个捕获组。

match.group(1) 返回第一个捕获组匹配的内容（方向编号字符串）。

为什么不是 group(0)？

group(0) 返回完整匹配的字符串（如 vertiefungsrichtung_25.php）。

group(1) 返回第一个括号捕获的内容（如 "25"）。

多捕获组示例

python
复制
# 假设正则表达式为：
pattern = re.compile(r'vertiefungsrichtung_(\d+)_(VR\d+)\.php')
# 匹配 URL: vertiefungsrichtung_2_VR25.php
match.group(1)  # "2"
match.group(2)  # "VR25"

soup就是找当前页面的所有信息并变成库。response请求并保存，soup = BeautifulSoup(response.text, 'html.parser') soup就是将信息保存到soup以启用其强大的功能


h1_tag = soup.find('h1') 就是找h1主标题
soup.find_all('a', href=re.compile(r'vertiefungsrichtung_\d+\.php'))就是找所有的，所以叫a_tag 用‘a’并且用正则regular express我请倾向于叫他规范表达，来显示其条件 for a in soup.find_all('a', href=re.compile(r'vertiefungsrichtung_\d+\.php'))就是遍历所有符合条件的标签。

最重要的就是要知道每一层在干嘛，想要加内容就到对应的层